In [1]:
from pytrends.request import TrendReq
import json
import pandas as pd   



In [3]:

class MyTrendReq(TrendReq):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def interest_by_region(self, resolution='COUNTRY', inc_low_vol=False,
                           inc_geo_code=False):
        """Request data from Google's Interest by Region section and return a dataframe"""

        # make the request
        region_payload = dict()

        if self.geo == '': 
            self.interest_by_region_widget['request']['resolution'] = resolution 
        elif self.geo == 'US' and resolution in ['DMA', 'CITY', 'REGION']: 
            self.interest_by_region_widget['request']['resolution'] = resolution 
        elif len(self.geo) == 2 and resolution in ['CITY', 'REGION']:
            self.interest_by_region_widget['request']['resolution'] = resolution        

        self.interest_by_region_widget['request'][
            'includeLowSearchVolumeGeos'] = inc_low_vol

        # convert to string as requests will mangle
        region_payload['req'] = json.dumps(
            self.interest_by_region_widget['request'])
        region_payload['token'] = self.interest_by_region_widget['token']
        region_payload['tz'] = self.tz

        # parse returned json
        req_json = self._get_data(
            url=TrendReq.INTEREST_BY_REGION_URL,
            method=TrendReq.GET_METHOD,
            trim_chars=5,
            params=region_payload,
        )
        df = pd.DataFrame(req_json['default']['geoMapData'])
        if (df.empty):
            return df

        # rename the column with the search keyword
        df = df[['geoName', 'value']].set_index(
            ['geoName']).sort_index()
        # split list columns into seperate ones, remove brackets and split on comma
        result_df = df['value'].apply(lambda x: pd.Series(
            str(x).replace('[', '').replace(']', '').split(',')))
        if inc_geo_code:
            result_df['geoCode'] = df['geoCode']

        # rename each column with its search term
        for idx, kw in enumerate(self.kw_list):
            result_df[kw] = result_df[idx].astype('int')
            del result_df[idx]

        return result_df



In [4]:
import datetime
import time

def getDateRangeFromWeek(p_year,p_week):

    firstdayofweek = datetime.datetime.strptime(f'{p_year}-W{int(p_week )- 1}-1', "%Y-W%W-%w").date()
    lastdayofweek = firstdayofweek + datetime.timedelta(days=6.9)
    return firstdayofweek, lastdayofweek


#Call function to get dates range 
calls = []
#for year in range(2017,2022, 1):
for week in range(1, 53):
    firstdate, lastdate =  getDateRangeFromWeek(str(2020),str(week))
    inter = str(firstdate)+'T00 ' + str(lastdate)+'T00'
    calls.append(inter)

calls[2]


'2020-01-13T00 2020-01-19T00'

In [5]:
pytrends = MyTrendReq(hl='en-US', tz=360, timeout=(15,30), retries=3, backoff_factor=0.2)
keywords = ['Elbil']
DK_geos = ["DK-81", "DK-84", "DK_82", "DK-85", "DK-83"]

#pytrends.related_queries()
pytrends.build_payload(kw_list = keywords, geo='DK', timeframe= '2017-01-02T00 2017-01-08T00')
df = pytrends.interest_by_region(resolution='CITY', inc_low_vol=True, inc_geo_code=False)
df


,Elbil
geoName,
Aalborg,47
Aarhus,35
Ballerup,42
Copenhagen,41
Fanø,27
Frederiksberg,45
Haderslev,38
Hellerup,72
Herning,40


In [6]:
trends = pd.DataFrame()
def get_trends(years, trends, weeks = [i+1 for i in range(52)]):
    if isinstance(years, int):
        years = [years]
    for year in years:    
        for week in weeks:
            try:
                firstdate, lastdate =  getDateRangeFromWeek(str(year),str(week))
                inter = str(firstdate)+'T00 ' + str(lastdate)+'T00'
                pytrends.build_payload(kw_list = keywords, geo='DK', timeframe= inter)
                df = pytrends.interest_by_region(resolution='CITY', inc_low_vol=True, inc_geo_code=False)
                df['interval'] = [inter]*len(df)
                df['week'] = [week]*len(df)
                df['year'] = [year]*len(df)
                print(week)
                trends = trends.append(df)
            except Exception as e:
                print(repr(e))
                continue
        time.sleep(60)
    return trends


In [7]:
car_trends = get_trends([2017,2018,2019,2020], trends)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52


In [9]:
car_trends_21 = get_trends([2021], pd.DataFrame(), weeks=[i+1 for i in range(42)])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42


In [10]:
car_trends.append([car_trends_21])
car_trends.index.name = 'City'
car_trends.reset_index(inplace=True)
car_trends

,City,Elbil,interval,week,year
0,Aalborg,31.0,2016-12-26T00 2017-01-01T00,1.0,2017.0
1,Aarhus,25.0,2016-12-26T00 2017-01-01T00,1.0,2017.0
2,Copenhagen,41.0,2016-12-26T00 2017-01-01T00,1.0,2017.0
3,Fanø,25.0,2016-12-26T00 2017-01-01T00,1.0,2017.0
4,Frederiksberg,38.0,2016-12-26T00 2017-01-01T00,1.0,2017.0
...,...,...,...,...,...
15370,Virum,60.0,2020-12-21T00 2020-12-27T00,52.0,2020.0
15371,Vojens,39.0,2020-12-21T00 2020-12-27T00,52.0,2020.0
15372,Vordingborg,40.0,2020-12-21T00 2020-12-27T00,52.0,2020.0
15373,Værløse,71.0,2020-12-21T00 2020-12-27T00,52.0,2020.0


In [14]:
car_trends.to_csv("trends_data.csv")